# Supermarket basket association pattern mining

In this question, we perform association pattern mining using the supermarket dataset `supermarket.arff` from the [Weka MOOC](https://www.cs.waikato.ac.nz/ml/weka/courses.html).


1. Load the data file `supermarket.arff` into a pandas data frame

2. Remove the following attributes 
  - Department* 
  - non host support 
  - Total 

3.  Select the Apriori algorithm and perform frequent itemset mining with minsup = 0.2 and minconf = 0.8 and find out: 
  - The numbers of frequent 2-itemsets, and 3-itemsets. 
  - The best three (2) rules with largest confidence. Examine these rules and describe them in your own words. 

4. The supermarket manager wishes to boost the sale of fruit and therefore the manager needs to know other itemsets most likely be purchased with fruit to make promotion decisions. 
  - Using the same minimum support and minimum confidence value. 
  - List the top three itemsets to report to the supermarket manager. 

5. Repeat task 3, but using the FP Growth algorithm instead.  
  - Compare the rules found. 
  - Are they consistent? 

## 0 Upgrade mlxtend
The default version of `mlxtend` on Google Colaborate is too old for this prac
so we must upgrade it. We want something that is at least version 0.18.
Note that code statements beginning with `!` are not python code, but system calls. If you are running this in a personal jupyterlab you might have to update this module a different way.

In [1]:

! pip install --upgrade 'mlxtend>=0.18'

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.3 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 32.6 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 455 kB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 6.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
# Check we have the right version
import mlxtend
print(mlxtend.__version__)

0.18.0


In [5]:
import pandas as pd
from scipy.io import arff
import urllib
import urllib.request
import numpy as np

## 1 Load the data file `supermarket.arff` into a pandas data frame

We did this in a previous prac: download the file into your working directory using `urrlib`, load it using `scipy`, and then convert to a `pandas` data frame. The file on the Weka website has a few problems that we need to work around, so I've provided a cleaned version of the data on [GitHub](https://raw.githubusercontent.com/PaulHancock/COMP5009_pracs/main/data/supermarket.arff).

In [6]:
data_url = 'https://raw.githubusercontent.com/PaulHancock/COMP5009_pracs/main/data/supermarket.arff'
file_name = 'supermarket.arff'
urllib.request.urlretrieve(data_url, file_name)

('supermarket.arff', <http.client.HTTPMessage at 0x7f817801eaf0>)

In [7]:
# load the data from arff format
data = arff.loadarff('supermarket.arff')
raw_df = pd.DataFrame(data[0])
# The data table is 1 and 0, but we want it to be boolean (true/false) so we convert
df = raw_df.astype(bool)

In [8]:
df.describe()

,department1,department2,department3,department4,department5,department6,department7,department8,department9,grocery misc,...,department208,department209,department210,department211,department212,department213,department214,department215,department216,total
count,4627,4627,4627,4627,4627,4627,4627,4627,4627,4627,...,4627,4627,4627,4627,4627,4627,4627,4627,4627,4627
unique,2,2,2,2,2,2,2,1,2,2,...,1,1,2,2,2,2,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
freq,3580,4496,4537,4543,4452,4625,4560,4627,4545,4449,...,4627,4627,4436,4420,4589,4605,4627,4627,4627,4627


## 2 Remove attributes 
Remove the following attributes as they have been deemed to be not-useful:
  - Department* 
  - non host support 
  - Total 


In [10]:
cols_to_drop = ['non host support', 'total']
# Instead of hand writing all the names that start with department, use a loop
for col in df.columns:
  if col.startswith('department'):
    cols_to_drop.append(col)
print("The folloiwing columns will be dropped:")
print(cols_to_drop)

The folloiwing columns will be dropped:
['non host support', 'total', 'department1', 'department2', 'department3', 'department4', 'department5', 'department6', 'department7', 'department8', 'department9', 'department11', 'department57', 'department70', 'department79', 'department80', 'department81', 'department88', 'department89', 'department98', 'department100', 'department101', 'department102', 'department107', 'department108', 'department109', 'department110', 'department111', 'department112', 'department113', 'department114', 'department116', 'department117', 'department118', 'department119', 'department120', 'department122', 'department123', 'department124', 'department125', 'department126', 'department127', 'department128', 'department129', 'department130', 'department137', 'department138', 'department139', 'department140', 'department141', 'department142', 'department143', 'department144', 'department145', 'department146', 'department147', 'department148', 'department149', 'depa

In [11]:
df = df.drop(columns=cols_to_drop)

In [15]:
# confirm we have dropped the columns by showing a summary, we should have 104 cols left, all with descriptive names.
df.describe()

,grocery misc,baby needs,bread and cake,baking needs,coupons,juice-sat-cord-ms,tea,biscuits,canned fish-meat,canned fruit,...,casks red wine,750ml white nz,750ml red nz,750ml white imp,750ml red imp,sparkling nz,sparkling imp,brew kits/accesry,port and sherry,ctrled label wine
count,4627,4627,4627,4627,4627,4627,4627,4627,4627,4627,...,4627,4627,4627,4627,4627,4627,4627,4627,4627,4627
unique,2,2,2,2,1,2,2,2,2,2,...,2,2,2,2,2,2,2,1,2,1
top,False,False,True,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,4449,4008,3330,2795,4627,2463,3731,2605,3686,3344,...,4576,4346,4536,4528,4530,4498,4604,4627,4602,4627


## 3 Select the Apriori algorithm 

Select the Apriori algorithm and perform frequent itemset mining with `minsup = 0.2` and `minconf = 0.8` and find out:

- The numbers of frequent 2-itemsets, and 3-itemsets.
- The best three rules with largest confidence. Examine these rules and describe them in your own words.

The `apriori` algorithm is found in the `mlxtend` package, so we import it along with the `association_rules` function.

In [16]:
from mlxtend.frequent_patterns import apriori, association_rules

In [17]:
ap_itemsets = apriori(df, 
                      min_support=0.2,
                      use_colnames=True)

Now that we have our itemsets we want to chose those with `2<=k<=3`.
This isn't explicitly stored within our dataframe so we'll make a new column which is just the value of `len(itemsets)`.

In [18]:
def find_k(row):
  """Return the number of items in the itemset"""
  return len(row.itemsets)

# Create a new column which counts the number of items in the itemset
ap_itemsets['k'] = ap_itemsets.apply(find_k, # Apply the function `find_k`
                                     axis=1) # apply the function to each row

In [21]:
k2_itemsets = np.sum(ap_itemsets['k'] == 2)
k3_itemsets = np.sum(ap_itemsets['k'] == 3)
k4_itemsets = np.sum(ap_itemsets['k'] == 4)
k5_itemsets = np.sum(ap_itemsets['k'] == 5)
print(f"There are {k2_itemsets} itemsets with k=2")
print(f"There are {k3_itemsets} itemsets with k=3")
print(f"There are {k4_itemsets} itemsets with k=4")
print(f"There are {k5_itemsets} itemsets with k=5")

There are 182 itemsets with k=2
There are 252 itemsets with k=3
There are 77 itemsets with k=4
There are 2 itemsets with k=5


In [24]:
# Now lets see the top 10 itemsets
ap_itemsets.sort_values(by = 'support', ascending = False).head(10)

,support,itemsets,k
0,0.719689,(bread and cake),1
28,0.640156,(fruit),1
29,0.639939,(vegetables),1
23,0.635185,(milk-cream),1
1,0.604063,(baking needs),1
12,0.587206,(frozen foods),1
3,0.563000,(biscuits),1
2,0.532310,(juice-sat-cord-ms),1
51,0.505079,"(bread and cake, milk-cream)",2
16,0.503566,(party snack foods),1


Note that the top 10 itemsets are all 1-itemsets. Is this surprising to you?

We use these itemsets to generate association rules with a minimum confidence of 0.8.

In [25]:
ap_rules = association_rules(ap_itemsets, 
                             metric='confidence', 
                             min_threshold=0.8)

In [26]:
ap_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(canned fruit),(bread and cake),0.277285,0.719689,0.224768,0.810600,1.126320,0.025208,1.479997
1,(jams-spreads),(bread and cake),0.276205,0.719689,0.221958,0.803599,1.116593,0.023177,1.427242
2,(margarine),(bread and cake),0.494489,0.719689,0.395721,0.800262,1.111956,0.039843,1.403396
3,(small goods),(bread and cake),0.241193,0.719689,0.201426,0.835125,1.160398,0.027843,1.700148
4,"(biscuits, baking needs)",(bread and cake),0.381241,0.719689,0.314675,0.825397,1.146880,0.040300,1.605419


Note that the rules above are not sorted by confidence. We should do that ourselves by using the `sort_values` function.

In [27]:
ap_rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
177,"(biscuits, vegetables, frozen foods, fruit)",(bread and cake),0.224552,0.719689,0.200778,0.894129,1.242383,0.039171,2.647667
139,"(biscuits, fruit, margarine)",(bread and cake),0.231900,0.719689,0.202723,0.874185,1.214670,0.035828,2.227955
132,"(biscuits, fruit, frozen foods)",(bread and cake),0.282905,0.719689,0.247028,0.873186,1.213282,0.043425,2.210406
138,"(biscuits, vegetables, milk-cream)",(bread and cake),0.267128,0.719689,0.232332,0.869741,1.208496,0.040083,2.151954
117,"(baking needs, margarine, fruit)",(bread and cake),0.244003,0.719689,0.212016,0.868911,1.207342,0.036410,2.138320
...,...,...,...,...,...,...,...,...,...
154,"(fruit, frozen foods, bread and cake)",(vegetables),0.334558,0.639939,0.268424,0.802326,1.253752,0.054328,1.821483
153,"(vegetables, frozen foods, bread and cake)",(fruit),0.334558,0.640156,0.268424,0.802326,1.253329,0.054255,1.820389
91,"(breakfast food, vegetables)",(fruit),0.275989,0.640156,0.221310,0.801879,1.252632,0.044634,1.816290
173,"(vegetables, frozen foods, milk-cream)",(fruit),0.285066,0.640156,0.228442,0.801365,1.251828,0.045955,1.811583


Now describe the first three that you see above in your own words.

## 4 Boost fruit sales
The supermarket manager wishes to boost the sale of fruit and therefore the manager needs to know other itemsets most likely be purchased with fruit to make promotion decisions. 
  - Using the same minimum support and minimum confidence value. 
  - List the top three itemsets to report to the supermarket manager. 

In [ ]:
# choose all the rules wihch have "fruit" (not canned fruit) as the consquent
fruit_rules = ap_rules[ap_rules.consequents == frozenset([?])]
# now sort based on confidence and report the top 3
fruit_rules.sort_values(?,
                        ascending=False).head(3)

## 5 FP-Growth
Repeat task 3, but using the FP Growth algorithm instead.  
  - Compare the rules found. 
  - Are they consistent? 

Import the `fpgrowth` function from our `mlxtend` module

In [ ]:
from mlxtend.frequent_patterns import fpgrowth


In [ ]:
fp_itemsets = fpgrowth(df,
                       min_support=?,
                       use_colnames=True)

In [ ]:
fp_rules = association_rules(fp_itemsets, 
                             metric='confidence', 
                             min_threshold=?)

There are a lot of rules, lets compare just the first 10 most confident rules.

In [ ]:
# Select the top 10 confident rules from each of our algorithms
fp_top_10 = fp_rules.sort_values('confidence', ascending=False).head(10)
ap_top_10 = ap_rules.sort_values('confidence', ascending=False).head(10)

In [ ]:
print("FP-Growth rules")
fp_top_10

In [ ]:
print("Apriori rules")
ap_top_10

Do the above tables agree?